# Who Got Hepatitis C

In [1]:
# Packages for classifier
import pandas as pd
import numpy as np 
import math 
import random 
import csv

# Packages for GLM
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import metrics
import seaborn as sns
sns.set(style='white')
sns.set(style='whitegrid', color_codes=True)
import matplotlib.pyplot as plt
import matplotlib
import statsmodels.api as sm
import scipy.stats as stats
from scipy.special import gammaln
import itertools
from itertools import chain
matplotlib.rcParams['font.size'] = 18

import time

# 1. Naive Bayes Classifier

In [2]:
def separate_by_class(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated


In [3]:
# function to calculate mean values
def mean(numbers):
    return sum(numbers)/len(numbers)

# function to calculate the sample standard deviation
def stdev(numbers):
    avg = mean(numbers)
    variance = np.var(numbers)*(len(numbers))/(len(numbers) - 1)
    return math.sqrt(variance)

# calculate the mean and ssd of attributes in one class/group
def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in  zip(*dataset)]
    del summaries[-1]
    return summaries


In [4]:
# for every class/group, calculate the mean and standard deviation
def summarize_by_class(dataset):
    separated = separate_by_class(dataset)
    summaries = {}
    keyList = list(separated.keys())
    for classValue in keyList:
        summaries[classValue] = summarize(separated[classValue])
    return summaries

In [5]:
# Use the pdf of Normal dist to calculate the probability
def calculate_probability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

In [6]:
# calculate the probability of every class/group
def calculate_class_probabilities(summaries, inputVector):
    probabilities = {}
    keyList = list(summaries.keys())
    for classValue in keyList:
        probabilities[classValue] = 1
        for i in range(len(summaries[classValue])):  # number of attributes
            mean, stdev = summaries[classValue][i]   # The ith attributes got from training
            x = inputVector[i]                       # the ith attributes from sample
            probabilities[classValue] *= calculate_probability(x, mean, stdev)
    return probabilities

In [7]:
def predict(summaries, inputVector):
    probabilities = calculate_class_probabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    # extract the label (class)
    keyList = list(probabilities.keys())
    # compare the probabilities of classes and select the best one
    for classValue in keyList:
        if bestLabel is None or probabilities[classValue] > bestProb:
            bestProb = probabilities[classValue]
            bestLabel = classValue
    return bestLabel

In [8]:
def get_predictions(summaries, testSet):
    # for every single row data, get the best label and record
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions

In [9]:
def get_accuracy(testSet, predictions):
    # number of successful predictions divided by the total observations
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet)))*100.0

In [11]:
def main():
    # import data
    filename = 'hcvM.csv'
    dataset = pd.read_csv(filename, header=None)
    dataset = np.array(dataset)
 
    # randomly divide the dataset into trainSet and testSet
    trainSize = int(len(dataset)*2/3) 
    randomIdx = [i for i in range(len(dataset))]
    random.shuffle(randomIdx)
    trainSet = []
    testSet = []
    trainSet.extend(dataset[idx,:] for idx in randomIdx[:trainSize])
    testSet.extend(dataset[idx,:] for idx in randomIdx[trainSize:])
 
    # compute the mdoel
    summaries = summarize_by_class(trainSet)
    #print(summaries)
 
    # test the model by testSet
    predictions = get_predictions(summaries, testSet)
    accuracy = get_accuracy(testSet, predictions)
    print(('Accuracy:{0}%').format(accuracy))

In [68]:
starttime = time.time()
if __name__ == '__main__':
    main()

print('That took {} seconds'.format(time.time() - starttime))

Accuracy:93.81443298969072%
That took 0.012001514434814453 seconds


# 2. GLM Regression (Logit)

In [49]:
def splitset(data):
    # extract the header
    headers = data.columns.values
    dataset = np.array(data)

    # Randomly shuffle the data rows and split into two sets
    trainSize = int(len(dataset)*2/3) 
    randomIdx = [i for i in range(len(dataset))]
    random.shuffle(randomIdx)
    trainSet = []
    testSet = []
    trainSet.extend(dataset[idx,:] for idx in randomIdx[:trainSize])
    testSet.extend(dataset[idx,:] for idx in randomIdx[trainSize:])

    trainSet = pd.DataFrame(trainSet, columns=headers)
    testSet = pd.DataFrame(testSet, columns=headers)
    return trainSet, testSet

In [74]:
starttime = time.time()

# data import
hcvR = pd.read_csv('hcvR.csv')
train_hcv, test_hcv = splitset(hcvR)

Ty = train_hcv[['Cate']]
Tx = train_hcv[['Age', 'ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT', 'SEX']]
Sy = test_hcv[['Cate']]
Sx = test_hcv[['Age', 'ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT', 'SEX']]

dataT = {'X': np.array(Tx), 'y': np.array(Ty)}
XT = np.insert(dataT['X'],0, values=np.ones(dataT['X'].shape[0]), axis=1)
dataS = {'X': np.array(Sx), 'y': np.array(Sy)}
XS = np.insert(dataS['X'],0, values=np.ones(dataS['X'].shape[0]), axis=1)

# OvR approach
lr = linear_model.LogisticRegression().fit(XT, dataT['y'])
print(metrics.accuracy_score(dataS['y'], lr.predict(XS)))

print('That took {} seconds'.format(time.time() - starttime))

0.9536082474226805
That took 0.025022268295288086 seconds


In [82]:
starttime = time.time()

# data import
hcvR = pd.read_csv('hcvR.csv')
train_hcv, test_hcv = splitset(hcvR)

Ty = train_hcv[['Cate']]
Tx = train_hcv[['Age', 'ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT', 'SEX']]
Sy = test_hcv[['Cate']]
Sx = test_hcv[['Age', 'ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT', 'SEX']]

dataT = {'X': np.array(Tx), 'y': np.array(Ty)}
XT = np.insert(dataT['X'],0, values=np.ones(dataT['X'].shape[0]), axis=1)
dataS = {'X': np.array(Sx), 'y': np.array(Sy)}
XS = np.insert(dataS['X'],0, values=np.ones(dataS['X'].shape[0]), axis=1)

# MvM approach
multi_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(XT, dataT['y'])
print(metrics.accuracy_score(dataS['y'], multi_lr.predict(XS)))

print('That took {} seconds'.format(time.time() - starttime))

0.9639175257731959
That took 0.19217467308044434 seconds
